In [ ]:
# pip install xgboost

In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, classification_report

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier

## 1. Data Preparation

In [2]:
# Read the database
result = pyreadr.read_r('../01_data/minorities_discrimination_survey.RData')  
print(result.keys())  

odict_keys(['min_dis'])


In [63]:
# Activate automatic conversion from R to pandas
df1 = result["min_dis"] # extract the pandas data frame for object df1

In [64]:
df1.head()

,za_nr,version,doi,country,DEGURBA,ALTURBA,C1,C2,C3,typint,...,dis5_4_other,dis12overall10,dis5overall9,redisOverall,res_stat,sec_res,SI03_2_H_stat,SI03_3_H,pweightadj,hweightadj
0,6703.0,1.0.0 (2020-07-29),doi:10.4232/1.13514,1.0,1.0,NaN,1.0,2.0,2.0,1.0,...,0.0,1.0,1.0,NaN,3.0,1.0,0.0,15.0,0.075758,0.115287
1,6703.0,1.0.0 (2020-07-29),doi:10.4232/1.13514,1.0,1.0,NaN,1.0,2.0,2.0,1.0,...,0.0,NaN,0.0,NaN,99.0,2.0,0.0,15.0,NaN,0.115287
2,6703.0,1.0.0 (2020-07-29),doi:10.4232/1.13514,1.0,1.0,NaN,1.0,3.0,2.0,1.0,...,0.0,1.0,1.0,NaN,5.0,2.0,0.0,16.0,0.075758,0.057643
3,6703.0,1.0.0 (2020-07-29),doi:10.4232/1.13514,1.0,1.0,NaN,1.0,3.0,2.0,1.0,...,0.0,NaN,0.0,NaN,99.0,2.0,0.0,16.0,NaN,0.057643
4,6703.0,1.0.0 (2020-07-29),doi:10.4232/1.13514,1.0,1.0,NaN,1.0,3.0,2.0,1.0,...,0.0,NaN,0.0,NaN,99.0,2.0,0.0,16.0,NaN,0.057643


In [65]:
# Remove repetitive columns
df1 = df1.drop(['version', 'doi', 'za_nr'], axis=1)

In [66]:
# Display information about the DataFrame
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77656 entries, 0 to 77655
Columns: 1280 entries, country to hweightadj
dtypes: float64(1277), object(3)
memory usage: 758.4+ MB


In [67]:
# Count missing values 
df1.isnull().sum()

country                    0
DEGURBA                    0
ALTURBA                43030
C1                     10860
C2                     11767
C3                     10860
typint                     0
IN02                       0
IN03                       0
numints                    0
IN05                   48074
IN06                   76823
IN07                   36764
IN08                   36764
IN10                   74831
resp                       0
Generation             49322
HH01                       0
HH02                       0
HH02_1                 77538
HH03                       0
HH04                   22450
HH05                   55969
HH06                   54841
HH07_1                 60088
HH07_2                 45822
HH07_3                 68030
HH07_4                 77574
HH08                   61109
HH09_1                 55477
HH10                   55477
HH10a                  71967
HH10b                  76691
HH10c                  73795
HH14          

## Subset (People actually looking for a Job)

In [68]:
# Filter the subset to include only people seeking for a job (EU05=1 or EU07=1)
df_subset_job_seekers = df1[(df1['EU05'] == 1) | (df1['EU07'] == 1)]

In [69]:
# Count the number of observations before filtering
before_filter_count = df1.shape[0]

# Count the number of observations after filtering
after_filter_count = df_subset_job_seekers.shape[0]

# Print the counts
print("Number of observations before filtering:", before_filter_count)
print("Number of observations after filtering for job seekers:", after_filter_count)

# Percentage of cases
round((df_subset_job_seekers.shape[0] / df1.shape[0])*100, 1)

Number of observations before filtering: 77656
Number of observations after filtering for job seekers: 13406


17.3

## 2. Construct the Target variable

In [131]:
# Define the target variable
discrimination_columns = ['EUD01_01', 'EUD01_02', 'EUD01_03', 'EUD01_04', 'EUD01_05', 'EUD01_06', 
                          'EUD01_07', 'EUD01_08']

# Define a function to count the discrimination level
def count_discrimination_level(row):
    discrimination_count = 0
    for col in discrimination_columns:
        # Check if discrimination occurred in each column 
        if row[col] == 1:
            discrimination_count += 1
    return discrimination_count

# Create a copy of the DataFrame
df_subset_copy = df_subset_job_seekers.copy()

# Create the discrimination level variable
df_subset_copy['discrimination_level'] = df_subset_copy.apply(count_discrimination_level, axis=1)

# Display the distribution of the discrimination level
print(df_subset_copy['discrimination_level'].value_counts().sort_index())

discrimination_level
0    8236
1    3021
2    1636
3     419
4      78
5      16
Name: count, dtype: int64


In [132]:
# But the variable EUD01_09 contains people that said that they haven't felt discriminated against on any ground when looking for work in the past 5 years
df_subset_job_seekers['EUD01_09'].value_counts()

EUD01_09
1.0    7940
0.0    5466
Name: count, dtype: int64

In [168]:
# Add to the count_discrimination_level funtion a condition to discrimination_count being 0 when the value in EUD01_09 is 1

# Define the target variable including 'EUD01_09'
discrimination_columns = ['EUD01_01', 'EUD01_02', 'EUD01_03', 'EUD01_04', 'EUD01_05', 'EUD01_06', 
                          'EUD01_07', 'EUD01_08', 'EUD01_09']

# Define a function to count the discrimination level
def count_discrimination_level(row):
    discrimination_count = 0
    for col in discrimination_columns:
        # Check if discrimination occurred in each column 
        if row[col] == 1:
            discrimination_count += 1
    
    # Check if the individual stated 1 in column 'EUD01_09'
    if row['EUD01_09'] == 1:
        discrimination_count = 0
    
    return discrimination_count

# Create a copy of the DataFrame
df_subset_copy = df_subset_job_seekers.copy()

# Create the discrimination level variable
df_subset_copy['discrimination_level'] = df_subset_copy.apply(count_discrimination_level, axis=1)

# Display the distribution of the discrimination level
print(df_subset_copy['discrimination_level'].value_counts().sort_index())

# Apparently they are included in our count of 0

discrimination_level
0    8236
1    3021
2    1636
3     419
4      78
5      16
Name: count, dtype: int64


## 3. Choose factors

In [148]:
# Count missing values in the subset dataframe
df_subset_copy.isnull().sum()

country                     0
DEGURBA                     0
ALTURBA                  9213
C1                       2501
C2                       2721
C3                       2501
typint                      0
IN02                        0
IN03                        0
numints                     0
IN05                     4776
IN06                    12967
IN07                     4337
IN08                     4337
IN10                    12028
resp                        0
Generation               4337
HH01                        0
HH02                        0
HH02_1                  13406
HH03                        0
HH04                        0
HH05                     7678
HH06                     6317
HH07_1                   3987
HH07_2                   3987
HH07_3                   7931
HH07_4                  13365
HH08                     9210
HH09_1                   9419
HH10                     9419
HH10a                   12648
HH10b                   13376
HH10c     

In [149]:
# Count missing values in the subset dataframe as a percentage
df_subset_copy.isna().sum() / len(df_subset_copy) * 100

country                   0.000000
DEGURBA                   0.000000
ALTURBA                  68.722960
C1                       18.655826
C2                       20.296882
C3                       18.655826
typint                    0.000000
IN02                      0.000000
IN03                      0.000000
numints                   0.000000
IN05                     35.625839
IN06                     96.725347
IN07                     32.351186
IN08                     32.351186
IN10                     89.721020
resp                      0.000000
Generation               32.351186
HH01                      0.000000
HH02                      0.000000
HH02_1                  100.000000
HH03                      0.000000
HH04                      0.000000
HH05                     57.272863
HH06                     47.120692
HH07_1                   29.740415
HH07_2                   29.740415
HH07_3                   59.160078
HH07_4                   99.694167
HH08                

In [169]:
# Remove columns with more than 90% missing values, 

# Calculate the percentage of missing values in each column
missing_percentage = (df_subset_copy.isnull().sum() / len(df_subset_copy)) * 100

# Identify columns with more than 90% missing values
columns_to_drop = missing_percentage[missing_percentage > 90].index

# Drop columns with more than 90% missing values
df_subset_copy = df_subset_copy.drop(columns=columns_to_drop)

# Print the DataFrame after dropping columns
print(df_subset_copy.head())

    country  DEGURBA  ALTURBA   C1   C2   C3  typint  IN02  IN03  numints  \
0       1.0      1.0      NaN  1.0  2.0  2.0     1.0   8.0   2.0      1.0   
12      1.0      1.0      NaN  NaN  NaN  NaN     2.0   8.0   1.0      1.0   
16      1.0      1.0      NaN  NaN  NaN  NaN     2.0   8.0   1.0      1.0   
17      1.0      1.0      NaN  NaN  NaN  NaN     2.0   8.0   1.0      1.0   
22      1.0      1.0      NaN  NaN  NaN  NaN     2.0   8.0   1.0      1.0   

    ...  dis5_4_other  dis12overall10  dis5overall9  res_stat  sec_res  \
0   ...           0.0             1.0           1.0       3.0      1.0   
12  ...           0.0             0.0           1.0       5.0      2.0   
16  ...           0.0             1.0           1.0       5.0      2.0   
17  ...           0.0             1.0           1.0       3.0      1.0   
22  ...           0.0             1.0           1.0       5.0      2.0   

    SI03_2_H_stat  SI03_3_H  pweightadj  hweightadj  discrimination_level  
0             0.

In [151]:
# Verify
df_subset_copy.isna().sum() / len(df_subset_copy) * 100

country                  0.000000
DEGURBA                  0.000000
ALTURBA                 68.722960
C1                      18.655826
C2                      20.296882
C3                      18.655826
typint                   0.000000
IN02                     0.000000
IN03                     0.000000
numints                  0.000000
IN05                    35.625839
IN07                    32.351186
IN08                    32.351186
IN10                    89.721020
resp                     0.000000
Generation              32.351186
HH01                     0.000000
HH02                     0.000000
HH03                     0.000000
HH04                     0.000000
HH05                    57.272863
HH06                    47.120692
HH07_1                  29.740415
HH07_2                  29.740415
HH07_3                  59.160078
HH08                    68.700582
HH09_1                  70.259585
HH10                    70.259585
HH14                    89.721020
HH24          

In [177]:
# Handle Missing Values
db_wms = df_subset_copy.copy()

In [178]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Select categorical variables (non-numerical)
categorical_cols = db_wms.columns.tolist()

# For categorical variables, impute using mode
categorical_imputer = SimpleImputer(strategy='most_frequent')
db_wms[categorical_cols] = categorical_imputer.fit_transform(db_wms)

# Impute missing values in 'age' variable using mean
age_imputer = SimpleImputer(strategy='mean')
db_wms['HH02'] = age_imputer.fit_transform(db_wms[['HH02']])

/Users/monserratlopez/miniconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/fixes.py:60: RuntimeWarning: invalid value encountered in not_equal
  return X != X
/Users/monserratlopez/miniconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/fixes.py:60: RuntimeWarning: invalid value encountered in not_equal
  return X != X


In [174]:
db_wms.head()

,country,DEGURBA,ALTURBA,C1,C2,C3,typint,IN02,IN03,numints,...,dis5_4_other,dis12overall10,dis5overall9,res_stat,sec_res,SI03_2_H_stat,SI03_3_H,pweightadj,hweightadj,discrimination_level
0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,8.0,2.0,1.0,...,0.0,1.0,1.0,3.0,1.0,0.0,15.0,0.075758,0.115287,2
12,1.0,1.0,1.0,1.0,4.0,2.0,2.0,8.0,1.0,1.0,...,0.0,0.0,1.0,5.0,2.0,0.0,19.0,0.075758,0.057643,0
16,1.0,1.0,1.0,1.0,4.0,2.0,2.0,8.0,1.0,1.0,...,0.0,1.0,1.0,5.0,2.0,0.0,17.0,0.075758,0.230573,0
17,1.0,1.0,1.0,1.0,4.0,2.0,2.0,8.0,1.0,1.0,...,0.0,1.0,1.0,3.0,1.0,0.0,14.0,0.075758,0.046115,3
22,1.0,1.0,1.0,1.0,4.0,2.0,2.0,8.0,1.0,1.0,...,0.0,1.0,1.0,5.0,2.0,0.0,10.0,0.075758,0.230573,0


## Split the data set

In [179]:
# Disccrimination level is our target variable 

# Remove string values in the dataset (IA07 has letters)
db_wms = db_wms.drop('IA07', axis=1)
db_wms = db_wms.drop('PB06_01', axis=1)

# Separate features (X) and target variable (y)
X = db_wms.drop('discrimination_level', axis=1)
y = db_wms['discrimination_level']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Random Forest for Feature Selection


In [180]:
# Apply Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

ValueError: could not convert string to float: 'Not on route'